In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local[*]").getOrCreate()

In [3]:
from pyspark.sql import functions as f
from pyspark.sql import Window

In [4]:
data = sparkSession.read.parquet("/data/sample264")

In [5]:
data.take(10)

[Row(userId=13065, trackId=944906, artistId=978428, timestamp=1501588527),
 Row(userId=101897, trackId=799685, artistId=989262, timestamp=1501555608),
 Row(userId=215049, trackId=871513, artistId=988199, timestamp=1501604269),
 Row(userId=309769, trackId=857670, artistId=987809, timestamp=1501540265),
 Row(userId=397833, trackId=903510, artistId=994595, timestamp=1501597615),
 Row(userId=501769, trackId=818149, artistId=994975, timestamp=1501577955),
 Row(userId=601353, trackId=958990, artistId=973098, timestamp=1501602467),
 Row(userId=710921, trackId=916226, artistId=972031, timestamp=1501611582),
 Row(userId=6743, trackId=801006, artistId=994339, timestamp=1501584964),
 Row(userId=152407, trackId=913509, artistId=994334, timestamp=1501571055)]

In [6]:
tracks = data.alias('df1').join(data.alias('df2'), 'userId') \
    .withColumn('diff', f.col('df1.timestamp') - f.col('df2.timestamp')) \
    .where((f.col('diff') > 0) & (f.col('diff') <= 420) & (f.col('df1.trackId') != f.col('df2.trackId'))) \
    .select(f.col('df1.trackId').alias('track1'), f.col('df2.trackId').alias('track2')) \
    .withColumn('id1', f.when((f.col('track1') < f.col('track2')), f.col('track1')).otherwise(f.col('track2'))) \
    .withColumn('id2', f.when((f.col('track1') < f.col('track2')), f.col('track2')).otherwise(f.col('track1'))) \
    .select('id1', 'id2') \
    .groupBy(f.col('id1'), f.col('id2')).count()

In [7]:
tracks.take(30)

[Row(id1=874523, id2=900026, count=1),
 Row(id1=812616, id2=834204, count=1),
 Row(id1=891623, id2=927170, count=15),
 Row(id1=800288, id2=944500, count=1),
 Row(id1=930224, id2=933768, count=47),
 Row(id1=814652, id2=884527, count=1),
 Row(id1=857076, id2=858940, count=1),
 Row(id1=829417, id2=892124, count=7),
 Row(id1=807087, id2=830876, count=1),
 Row(id1=822237, id2=867350, count=2),
 Row(id1=824673, id2=954186, count=4),
 Row(id1=805959, id2=901935, count=12),
 Row(id1=915099, id2=936935, count=3),
 Row(id1=846674, id2=872475, count=1),
 Row(id1=822971, id2=831261, count=4),
 Row(id1=809695, id2=842209, count=1),
 Row(id1=865876, id2=965934, count=1),
 Row(id1=855221, id2=871513, count=7),
 Row(id1=836384, id2=860385, count=2),
 Row(id1=846624, id2=940951, count=9),
 Row(id1=802640, id2=953737, count=16),
 Row(id1=893411, id2=937511, count=1),
 Row(id1=799143, id2=847678, count=1),
 Row(id1=803513, id2=825398, count=1),
 Row(id1=874318, id2=887900, count=1),
 Row(id1=864690, id2=

In [8]:
def norm(df, key1, key2, field, n): 
    
    window = Window.partitionBy(key1).orderBy(f.col(field).desc())
        
    topsDF = df.withColumn('row_number', f.row_number().over(window)) \
        .filter(f.col('row_number') <= n) \
        .drop(f.col('row_number')) 
        
    tmpDF = topsDF.groupBy(f.col(key1)).agg(f.col(key1), f.sum(f.col(field)).alias('sum_' + field))
   
    normalizedDF = topsDF.join(tmpDF, key1, 'inner') \
        .withColumn('norm_' + field, f.col(field) / f.col('sum_' + field)) \
        .cache()

    return normalizedDF

In [9]:
tracksNorm = norm(tracks, 'id1', 'id2', 'count', 40) \
        .select('id1', 'id2', 'norm_count')

In [10]:
tracksNorm.take(50)

[Row(id1=798477, id2=883244, norm_count=1.0),
 Row(id1=798692, id2=898823, norm_count=1.0),
 Row(id1=800467, id2=855206, norm_count=1.0),
 Row(id1=801701, id2=920990, norm_count=1.0),
 Row(id1=802599, id2=908754, norm_count=0.03571428571428571),
 Row(id1=802599, id2=937714, norm_count=0.03571428571428571),
 Row(id1=802599, id2=811513, norm_count=0.03571428571428571),
 Row(id1=802599, id2=929402, norm_count=0.03571428571428571),
 Row(id1=802599, id2=924227, norm_count=0.03571428571428571),
 Row(id1=802599, id2=901687, norm_count=0.03571428571428571),
 Row(id1=802599, id2=860294, norm_count=0.03571428571428571),
 Row(id1=802599, id2=880642, norm_count=0.03571428571428571),
 Row(id1=802599, id2=920627, norm_count=0.03571428571428571),
 Row(id1=802599, id2=843219, norm_count=0.03571428571428571),
 Row(id1=802599, id2=892457, norm_count=0.03571428571428571),
 Row(id1=802599, id2=823001, norm_count=0.03571428571428571),
 Row(id1=802599, id2=899859, norm_count=0.03571428571428571),
 Row(id1=8

In [11]:
window = Window.orderBy(f.col('norm_count').desc())
    
TrackList = tracksNorm.withColumn('position', f.rank().over(window)) \
    .filter(f.col('position') < 40) \
    .orderBy(f.col('id1'), f.col('id2')) \
    .select('id1', 'id2') \
    .take(40)

In [12]:
for val in TrackList:
    print "%s %s" % val

798256 923706
798319 837992
798322 876562
798331 827364
798335 840741
798374 816874
798375 810685
798379 812055
798380 840113
798396 817687
798398 926302
798405 867217
798443 905923
798457 918918
798460 891840
798461 940379
798470 840814
798474 963162
798477 883244
798485 955521
798505 905671
798545 949238
798550 936295
798626 845438
798691 818279
798692 898823
798702 811440
798704 937570
798725 933147
798738 894170
798745 799665
798782 956938
798801 950802
798820 890393
798833 916319
798865 962662
798931 893574
798946 946408
799012 809997
799024 935246
